## Setup

### Imports etc

In [449]:
import gzip
import json
from typing import List
from collections import Counter, defaultdict
from tqdm import tqdm
import os
import inspect_ai
from openai import OpenAI, AsyncOpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv
import torch as t
import subprocess
import contextlib
import shutil
import ast
import copy
from openai import OpenAI
from datetime import datetime
from dataclasses import dataclass

In [450]:
from inspect_ai import Task, task
from inspect_ai.dataset import Sample, hf_dataset
from inspect_ai.util import ExecResult, sandbox
from inspect_ai.scorer import CORRECT, INCORRECT, Score, Scorer, Target, accuracy, scorer, stderr
from inspect_ai.solver import TaskState, generate
from inspect_ai.model import get_model
from inspect_ai.log import read_eval_log
import re

In [451]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])
client = OpenAI()
async_client = AsyncOpenAI()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


[07/30/25 22:06:19] WARNING  Note: Environment variable`HF_TOKEN` is set and is the current active    ]8;id=419692;file:///root/miniconda3/envs/srf-env/lib/python3.11/site-packages/huggingface_hub/_login.py\_login.py]8;;\:]8;id=295640;file:///root/miniconda3/envs/srf-env/lib/python3.11/site-packages/huggingface_hub/_login.py#415\415]8;;\
                             token independently from the token you've just configured.                            

In [452]:
IMPORT_HELPER = {
    "python": [
        "import math",
        "import re",
        "import sys",
        "import copy",
        "import datetime",
        "import itertools",
        "import collections",
        "import heapq",
        "import statistics",
        "import functools",
        "import hashlib",
        "import numpy",
        "import numpy as np",
        "import string",
        "from typing import *",
        "from collections import *",
    ],
    "go"    : [
        "math",
        "strings",
        "fmt",
        "strconv",
        "time",
        "bytes",
        "regexp",
        "sort",
        "math/rand",
        "crypto/md5",
        "encoding/hex",
    ],
    "cpp"   : [
        "#include<stdlib.h>",
        "#include<algorithm>",
        "#include<math.h>",
        "#include<stdio.h>",
        "#include<vector>",
        "#include<string>",
        "#include<climits>",
        "#include<cstring>",
        "#include<iostream>",
        "#include <numeric>",
        "#include <sstream>",
        "#include <stack>",
        "#include <cctype>",
        "#include <set>",
        "#include <unordered_set>",
        "#include <iomanip>",
    ],
}

# instruction prepended to code problem
HUMANEVAL_INSTRUCTION = """
Read the following function signature and docstring, and fully implement
the function described. Include a function signature, that's exactly the 
same as the one provided in the prompt. Do NOT include any explanations. 
Also do NOT include the `main` function\n
"""

# ''' 
# Read the following function signature and docstring, and fully implement
# the function described. Your response should only contain the code for
# this function. Do NOT include block comments.  
# IF you have helper functions, ONLY include them below the primary function. 
# Never include them above. You are also welcome to nest them within the primary function.
# Include forward declarations\n'''

CODE_EXTRACTION_INSTRUCTION = """
Here is a section of code. Remove any leading package names and comments. 
Leave any indentation as it is (ie. don't remove leading whitespace for any line).
Also leave any import statements, if present.\n
"""

# EVERYTHING BUT GO
# """
# Here is a section of code. Remove any leading package names, import statements, and comments. 
# Leave any indentation as it is (ie. don't remove leading whitespace for any line).\n
# """

# """
# Here is a section of code. Follow these instructions to extract the function body.
# First, if present, remove any leading package names, import statements, and comments. 
# Next, remove the function signature of the first function you see. 
# If there is a `main` function present, delete that entire function. 
# However, if there are any other functions present, leave them as they are.
# Leave any indentation as it is (ie. don't remove leading whitespace for any line).

# Your response should only contain the remaining block of code. \n
# """ 

LANG_PREFIX = {
    "cpp"          : "// language: C++",
    "java"         : "// language: Java",
    "js"           : "// language: JavaScript",
    "javascript"   : "// language: JavaScript",
    "go"           : "// language: Go",
    "python"       : "# language: Python",
}

In [453]:
llama_model = get_model(
        'hf/meta-llama/Llama-3.1-8B-Instruct', 
        device='auto',
        torch_dtype="auto"
)

qwen_coder = get_model(
    'hf/Qwen/Qwen2.5-Coder-7B-Instruct',
    device="auto",
    torch_dtype="auto",
)

# deepseek_coder = get_model(
#     'hf/deepseek-ai/deepseek-coder-6.7b-instruct',
#     device='auto',
#     torch_dtype='auto',
# )

### Few-Shot Prompting

In [454]:
py_prompt_template = """
```python

    ```
    """

py_prompt_1 = """
```python
import math

def right_angle_triangle(a, b, c):
    '''
    Given the lengths of the three sides of a triangle. Return True if the three
    sides form a right-angled triangle, False otherwise.
    A right-angled triangle is a triangle in which one angle is right angle or 
    90 degree.
    Example:
    right_angle_triangle(3, 4, 5) == True
    right_angle_triangle(1, 2, 3) == False
    '''
    sides = sorted([a, b, c])
    return math.isclose(sides[0]**2 + sides[1]**2, sides[2]**2)
    ```
    """

py_response_1 = """
```python
    sides = sorted([a, b, c])
    return math.isclose(sides[0]**2 + sides[1]**2, sides[2]**2)
    ```
    """

py_prompt_2 = """
```python
def sorted_list_sum(lst):
    '''
    Deletes strings with odd lengths from a list and returns the resulted list 
    sorted in ascending order by string length and then alphabetically.

    Args:
        lst (list): A list of strings.

    Returns:
        list: A list of strings with odd lengths removed, sorted by length and then alphabetically.
    '''
    # Filter out strings with odd lengths
    even_length_strings = [string for string in lst if len(string) % 2 == 0]
    
    # Sort the list first by length and then alphabetically
    sorted_list = sorted(even_length_strings, key=lambda x: (len(x), x))
    
    return sorted_list
    ```
    """

py_response_2 = """
```python
    # Filter out strings with odd lengths
    even_length_strings = [string for string in lst if len(string) % 2 == 0]
    
    # Sort the list first by length and then alphabetically
    sorted_list = sorted(even_length_strings, key=lambda x: (len(x), x))
    
    return sorted_list
    ```
    """

py_prompt_3 = '''
```python
def words_in_sentence(sentence):
    ''''''
    Returns a string containing the words from the original sentence whose lengths are prime numbers.
    
    Args:
        sentence (str): A string representing a sentence.
    
    Returns:
        str: A string containing the words from the original sentence whose lengths are prime numbers.
    """
    def is_prime(n):
        """Checks if a number is prime."""
        if n < 2:
            return False
        for i in range(2, int(n ** 0.5) + 1):
            if n % i == 0:
                return False
        return True

    words = sentence.split()
    return ' '.join(word for word in words if is_prime(len(word)))
    ```
    '''

py_response_3 = '''
```python
    def is_prime(n):
        """Checks if a number is prime."""
        if n < 2:
            return False
        for i in range(2, int(n ** 0.5) + 1):
            if n % i == 0:
                return False
        return True

    words = sentence.split()
    return ' '.join(word for word in words if is_prime(len(word)))
    ```
    '''

py_prompt_4 = '''
```python
    """Evaluate whether the given number n can be written as the sum of exactly 4 positive even numbers
    Example
    is_equal_to_sum_even(4) == False
    is_equal_to_sum_even(6) == False
    is_equal_to_sum_even(8) == True
    """
    # The smallest sum of 4 positive even numbers (2 + 2 + 2 + 2) is 8
    if n < 8:
        return False
    
    # Since the sum of even numbers is even, n must be even
    if n % 2 != 0:
        return False
    
    # If n is at least 8 and is even, it can be expressed as the sum of 4 positive even numbers
    return True
    ```
    '''

py_response_4 = '''
```python
    # The smallest sum of 4 positive even numbers (2 + 2 + 2 + 2) is 8
    if n < 8:
        return False
    
    # Since the sum of even numbers is even, n must be even
    if n % 2 != 0:
        return False
    
    # If n is at least 8 and is even, it can be expressed as the sum of 4 positive even numbers
    return True
    ```
    '''

py_prompt_0 = """
```python
    txt = txt.rstrip()  # Remove trailing spaces
    if not txt:  # Check if the string is empty after stripping
        return False
    last_char = txt[-1]  # Get the last character of the string
    return last_char.isalpha() and (len(txt) == 1 or txt[-2] == ' ')  # Check conditions
    ```
    """

py_response_0 = """
```python
    txt = txt.rstrip()  # Remove trailing spaces
    if not txt:  # Check if the string is empty after stripping
        return False
    last_char = txt[-1]  # Get the last character of the string
    return last_char.isalpha() and (len(txt) == 1 or txt[-2] == ' ')  # Check conditions
    ```
    """


num_prompts = 5

PYTHON_PROMPT = [{
        'role': 'developer',
        'content': CODE_EXTRACTION_INSTRUCTION,
    }]

for i in range(num_prompts):
    user_msg = {
        'role': 'user',
        'content': globals()[f'py_prompt_{i}'],
    }
    PYTHON_PROMPT.append(user_msg)

    assistant_msg = {
        'role': 'assistant',
        'content': globals()[f'py_response_{i}']
    }
    PYTHON_PROMPT.append(assistant_msg)


In [455]:
js_prompt_template = """
```javascript

    ```
    """

js_prompt_0 = """
```javascript
const evenOddCount = (num) => {
    let evenCount = 0;
    let oddCount = 0;

    // Convert the number to a string and iterate through each digit
    for (let digit of Math.abs(num).toString()) {
        if (digit % 2 === 0) {
            evenCount++;
        } else {
            oddCount++;
        }
    }

    return [evenCount, oddCount];
};
    ```
    """

js_response_0 = """
```javascript
    let evenCount = 0;
    let oddCount = 0;

    // Convert the number to a string and iterate through each digit
    for (let digit of Math.abs(num).toString()) {
        if (digit % 2 === 0) {
            evenCount++;
        } else {
            oddCount++;
        }
    }

    return [evenCount, oddCount];
};
    ```
    """

js_prompt_1 = """
```javascript
/**
 * Calculates the Brazilian factorial of a given integer n.
 * The Brazilian factorial is defined as n! * (n-1)! * (n-2)! * ... * 1!
 * @param {number} n - The input integer.
 * @returns {number} The Brazilian factorial of n.
 */
const specialFactorial = (n) => {
  let result = 1;
  for (let i = n; i >= 1; i--) {
    result *= factorial(i);
  }
  return result;
};

/**
 * Helper function to calculate the factorial of a number.
 * @param {number} n - The input integer.
 * @returns {number} The factorial of n.
 */
const factorial = (n) => {
  if (n === 0 || n === 1) {
    return 1;
  } else {
    return n * factorial(n - 1);
  }
};
    ```
    """

js_response_1 = """
```javascript
  let result = 1;
  for (let i = n; i >= 1; i--) {
    result *= factorial(i);
  }
  return result;
};

const factorial = (n) => {
  if (n === 0 || n === 1) {
    return 1;
  } else {
    return n * factorial(n - 1);
  }
};
    ```
    """

js_prompt_2 = """
```javascript
const fixSpaces = (text) => {
  /**
   * Replaces all spaces in the input string with underscores, 
   * and if a string has more than 2 consecutive spaces, 
   * then replaces all consecutive spaces with a single hyphen.
   *
   * @param {string} text The input string to be processed.
   * @returns {string} The processed string with spaces replaced.
   */
  return text
    .replace(/ +/g, (match) => match === ' ' ? '_' : '-')
    .replace(/-/g, '_');
};
    ```
    """

js_response_2 = """
```javascript
  return text
    .replace(/ +/g, (match) => match === ' ' ? '_' : '-')
    .replace(/-/g, '_');
};
    ```
    """

js_prompt_3 = """
```javascript
/**
 * Checks if the second word or any of its rotations is a substring in the first word.
 *
 * @param {string} a - The main string to check.
 * @param {string} b - The substring to search for.
 * @returns {boolean} True if the second word or any of its rotations is a substring in the first word, false otherwise.
 */
const cycpatternCheck = (a, b) => {
  const rotations = [b];
  for (let i = 1; i < b.length; i++) {
    rotations.push(b.slice(i) + b.slice(0, i));
  }
  return a.includes(rotations.join('|')) || a.includes(b);
};
    ```
    """

js_response_3 = """
```javascript
  const rotations = [b];
  for (let i = 1; i < b.length; i++) {
    rotations.push(b.slice(i) + b.slice(0, i));
  }
  return a.includes(rotations.join('|')) || a.includes(b);
};
    ```
    """


num_prompts = 4

JS_PROMPT = [{
        'role': 'developer',
        'content': CODE_EXTRACTION_INSTRUCTION,
    }]

for i in range(num_prompts):
    user_msg = {
        'role': 'user',
        'content': globals()[f'js_prompt_{i}'],
    }
    JS_PROMPT.append(user_msg)

    assistant_msg = {
        'role': 'assistant',
        'content': globals()[f'js_response_{i}']
    }
    JS_PROMPT.append(assistant_msg)


In [456]:
cpp_prompt_template = """
```cpp

    ```
    """

cpp_prompt_0 = """
```cpp
int can_arrange(vector<int> arr) {
    int largest_index = -1;
    
    for (int i = 1; i < arr.size(); i++) {
        if (arr[i] < arr[i - 1]) {
            largest_index = i;
        }
    }
    
    return largest_index;
}
    ```
    """

cpp_response_0 = """
```cpp
    int largest_index = -1;
    
    for (int i = 1; i < arr.size(); i++) {
        if (arr[i] < arr[i - 1]) {
            largest_index = i;
        }
    }
    
    return largest_index;
}
    ```
    """

cpp_prompt_1 = """
```cpp
#include<stdio.h>
using namespace std;

bool is_equal_to_sum_even(int n) {
    // The minimum sum of 4 positive even numbers (2 + 2 + 2 + 2) is 8
    if (n < 8) {
        return false;
    }
    // Additionally, any even number 8 or greater can be expressed as the sum of 4 positive even numbers
    return n % 2 == 0; // Ensure n is even, which it must be to be a sum of evens
}
    ```
    """

cpp_response_1 = """
```cpp
    // The minimum sum of 4 positive even numbers (2 + 2 + 2 + 2) is 8
    if (n < 8) {
        return false;
    }
    // Additionally, any even number 8 or greater can be expressed as the sum of 4 positive even numbers
    return n % 2 == 0; // Ensure n is even, which it must be to be a sum of evens
}
    ```
    """

cpp_prompt_2 = """
```cpp
    string strongest_extension;
    int highest_strength = INT_MIN;

    for (const string& ext : extensions) {
        int CAP = 0;
        int SM = 0;

        for (char c : ext) {
            if (isupper(c)) {
                CAP++;
            } else if (islower(c)) {
                SM++;
            }
        }

        int strength = CAP - SM;

        if (strength > highest_strength) {
            highest_strength = strength;
            strongest_extension = ext;
        }
    }

    return class_name + "." + strongest_extension;
}
    ```
    """

cpp_response_2 = """
```cpp
    string strongest_extension;
    int highest_strength = INT_MIN;

    for (const string& ext : extensions) {
        int CAP = 0;
        int SM = 0;

        for (char c : ext) {
            if (isupper(c)) {
                CAP++;
            } else if (islower(c)) {
                SM++;
            }
        }

        int strength = CAP - SM;

        if (strength > highest_strength) {
            highest_strength = strength;
            strongest_extension = ext;
        }
    }

    return class_name + "." + strongest_extension;
}
    ```
    """

cpp_prompt_3 = """
```cpp
    // To check if the triangle is a right-angled triangle,
    // we will use the Pythagorean theorem.
    // We first need to find the longest side and ensure
    // that the sum of the squares of the other two sides
    // equals the square of the longest side.
    
    // Sort the sides to simplify comparisons
    float max_side = fmax(a, fmax(b, c));
    float side1, side2;

    if (max_side == a) {
        side1 = b;
        side2 = c;
    } else if (max_side == b) {
        side1 = a;
        side2 = c;
    } else {
        side1 = a;
        side2 = b;
    }

    // Check the Pythagorean theorem
    return (max_side * max_side) == (side1 * side1 + side2 * side2);
}
    ```
    """

cpp_response_3 = """
```cpp
    // Sort the sides to simplify comparisons
    float max_side = fmax(a, fmax(b, c));
    float side1, side2;

    if (max_side == a) {
        side1 = b;
        side2 = c;
    } else if (max_side == b) {
        side1 = a;
        side2 = c;
    } else {
        side1 = a;
        side2 = b;
    }

    // Check the Pythagorean theorem
    return (max_side * max_side) == (side1 * side1 + side2 * side2);
}
    ```
    """

cpp_prompt_4 = """
```cpp
long long factorial(int n) {
    long long result = 1;
    for (int i = 2; i <= n; ++i) {
        result *= i;
    }
    return result;
}

long long special_factorial(int n) {
    long long result = 1;
    for (int i = 1; i <= n; ++i) {
        result *= factorial(i);
    }
    return result;
}
    ```
    """

cpp_response_4 = """
```cpp
    long long result = 1;
    for (int i = 2; i <= n; ++i) {
        result *= i;
    }
    return result;
}

long long special_factorial(int n) {
    long long result = 1;
    for (int i = 1; i <= n; ++i) {
        result *= factorial(i);
    }
    return result;
}
    ```
    """

cpp_prompt_5 = """
```cpp
#include <iostream>
#include <sstream>
#include <vector>
#include <string>

using namespace std;

// Function to check if a number is prime
bool isPrime(int n) {
    if (n <= 1) return false;
    for (int i = 2; i * i <= n; i++) {
        if (n % i == 0) return false;
    }
    return true;
}

string words_in_sentence(string sentence) {
    stringstream ss(sentence);
    string word;
    vector<string> result;

    // Read words from the sentence
    while (ss >> word) {
        // Check if the length of the word is prime
        if (isPrime(word.length())) {
            result.push_back(word);
        }
    }

    // Join the result words into a single string
    string output;
    for (size_t i = 0; i < result.size(); i++) {
        output += result[i];
        if (i < result.size() - 1) {
            output += " "; // Add space between words
        }
    }

    return output;
}
    ```
    """

cpp_response_5 = """
```cpp
    if (n <= 1) return false;
    for (int i = 2; i * i <= n; i++) {
        if (n % i == 0) return false;
    }
    return true;
}

string words_in_sentence(string sentence) {
    stringstream ss(sentence);
    string word;
    vector<string> result;

    // Read words from the sentence
    while (ss >> word) {
        // Check if the length of the word is prime
        if (isPrime(word.length())) {
            result.push_back(word);
        }
    }

    // Join the result words into a single string
    string output;
    for (size_t i = 0; i < result.size(); i++) {
        output += result[i];
        if (i < result.size() - 1) {
            output += " "; // Add space between words
        }
    }

    return output;
}
    ```
    """


num_prompts = 6

CPP_PROMPT = [{
        'role': 'developer',
        'content': CODE_EXTRACTION_INSTRUCTION,
    }]

for i in range(num_prompts):
    user_msg = {
        'role': 'user',
        'content': globals()[f'cpp_prompt_{i}'],
    }
    CPP_PROMPT.append(user_msg)

    assistant_msg = {
        'role': 'assistant',
        'content': globals()[f'cpp_response_{i}']
    }
    CPP_PROMPT.append(assistant_msg)

### Language Setup

In [457]:
# JavaScript
#     apt-get update
#     apt-get install -y curl
#     curl -fsSL https://deb.nodesource.com/setup_lts.x | bash -
#     apt-get install -y nodejs
#     verify installation: `node -v` // `npm -v` // `node -e "console.log('Node.js is working')"

In [458]:
# Java
#     apt-get update
#     apt-get install -y openjdk-21-jdk
# Verify with:  
#     java -version
#     javac -version

In [459]:
# Go
#     cd ~/
#     GO_VERSION=1.24.5
#     wget https://go.dev/dl/go${GO_VERSION}.linux-amd64.tar.gz
#     rm -rf /usr/local/go
#     tar -C /usr/local -xzf go${GO_VERSION}.linux-amd64.tar.gz
#     echo 'export PATH=$PATH:/usr/local/go/bin' >> ~/.bashrc
#     export PATH=$PATH:/usr/local/go/bin
#     go version

In [460]:
# C++
#     apt-get update
#     apt-get install -y g++
#     apt-get install -y build-essential
#     apt-get install libboost-all-dev -y
#     apt-get install libssl-dev


## Data Processing

In [370]:
def stream_jsonl_all(filename: str):
    results = []
    fp = gzip.open(open(filename, "rb"), "rt")
    for line in fp:
        if any(not x.isspace() for x in line):
            results.append(json.loads(line))
    fp.close()

    return results

In [371]:
python_content = stream_jsonl_all('data/python_data.gz')
cpp_content = stream_jsonl_all('data/cpp_data.gz')
go_content = stream_jsonl_all('data/go_data.gz')
java_content = stream_jsonl_all('data/java_data.gz')
js_content = stream_jsonl_all('data/js_data.gz')
content = [python_content, cpp_content, go_content, java_content, js_content]

In [372]:
generations = stream_jsonl_all('data/python_generations.gz')
generations[0]['generation']

'    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n'

In [373]:
for lang in content:
    print(lang[0].keys())

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])
dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])
dict_keys(['task_id', 'prompt', 'import', 'docstring', 'declaration', 'canonical_solution', 'test', 'test_setup', 'example_test'])
dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])
dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])


## Eval Pipeline

### Extract Code

In [461]:
def identify_codeblock(completion: str) -> str:
    pattern_1 = re.compile(r"```(?:python|javascript|java|cpp|go)\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)

    if matches == []:
        return completion
    else:
        return matches[0]

In [462]:
async def remove_signature(completion, lang):
    prompt = CODE_EXTRACTION_INSTRUCTION + completion

    response = await async_client.responses.create(
        model='gpt-4.1-mini',
        input=prompt,
    )

    text_out = response.output[-1].content[0].text
    return text_out

### Languages

#### CPP

In [487]:
def balance_brackets_cpp(completion):
    if '{' in completion[:3]:
        completion = completion.replace('{', "", 1)

    difference = completion.count('{') - completion.count('}') 
    if difference == 0: 
        completion += '}'
        difference -= 1
    if difference != -1: 
        print('brackets ain\'t balancing')
    
    return completion

async def find_code_cpp(completion: str, lang: str) -> str:
    processed = await remove_signature(completion, lang)
    processed = identify_codeblock(processed)
    # processed = balance_brackets_cpp(processed)

    return processed

In [488]:
def get_final_cpp(state, completion):
    imports = ''
    for s in IMPORT_HELPER['cpp']:
        if s not in state.metadata['prompt']:
            imports += s + '\n'
    
    prompt = state.metadata['prompt']
    declaration = state.metadata['declaration']
    header = declaration.strip().split('\n')[-1]
    updated_prompt = ''.join(prompt.split(header)[:-1])

    code = imports + "\n" + updated_prompt + completion + "\n" + state.metadata['test']
    
    return code

#### Go

In [489]:
def balance_brackets_go(completion):
    if '{' in completion[:3]:
        completion = completion.replace('{', "", 1)

    difference = completion.count('{') - completion.count('}') 
    if difference == 0: 
        completion += '\n}'
        difference -= 1
    if difference != -1: 
        print('brackets ain\'t balancing')
    
    return completion

async def find_code_go(completion: str, lang: str) -> str:
    processed = await remove_signature(completion, lang)
    processed = identify_codeblock(processed)
    # processed = balance_brackets_go(processed)

    return processed

In [490]:
def get_final_go(state, completion):
    import_string = state.metadata['import']
    prompt = state.metadata['prompt'].replace(import_string, '')
    prompt = ''.join(prompt.split('func ')[:-1])

    test = state.metadata['test']
    test_setup = state.metadata['test_setup']
    other_pkgs = []

    for pkg in IMPORT_HELPER['go']:
        if pkg not in test_setup:
            p = pkg.split('/')[-1]
            if p + '.' in completion:    
                other_pkgs.append(f"\"{pkg}\"")
    if other_pkgs:
        import_other_pkgs = "import (\n" + "    ".join([p + "\n" for p in other_pkgs]) + ")"
        final_code = test_setup + "\n" + import_other_pkgs + "\n" + prompt + completion + "\n" + test
    else:
        final_code = test_setup + "\n" + prompt + completion + "\n" + test

    return final_code

In [491]:
def test_fxn(snippet, completion):
    import_string = snippet['import']
    prompt = snippet['prompt']
    prompt = ''.join(prompt.split('func ')[:-1])

    test = snippet['test']
    test_setup = snippet['test_setup']
    other_pkgs = []

    for pkg in IMPORT_HELPER['go']:
        if pkg not in test_setup:
            p = pkg.split('/')[-1]
            if p + '.' in completion:    
                other_pkgs.append(f"\"{pkg}\"")
    
    if other_pkgs:
        import_other_pkgs = "import (\n" + "    ".join([p + "\n" for p in other_pkgs]) + ")"
        final_code = test_setup + "\n" + import_other_pkgs + "\n" + prompt + completion + "\n" + test
    else:
        final_code = test_setup + "\n" + prompt + completion + "\n" + test

    return final_code

completion = """import (
    "crypto/md5"
    "encoding/hex"
    "errors"
)

func StringToMd5(text string) interface{} {
    if text == "" {
        return nil
    }

    hash := md5.New()
    hash.Write([]byte(text))
    md5Bytes := hash.Sum(nil)
    md5String := hex.EncodeToString(md5Bytes)

    return md5String
}"""

out = test_fxn(go_content[162], completion)
# print(out)

In [492]:
go_content[0]

{'task_id': 'Go/0',
 'prompt': 'import (\n    "math"\n)\n\n// Check if in given list of numbers, are any two numbers closer to each other than given threshold.\n// >>> HasCloseElements([]float64{1.0, 2.0, 3.0}, 0.5)\n// false\n// >>> HasCloseElements([]float64{1.0, 2.8, 3.0, 4.0, 5.0, 2.0}, 0.3)\n// true\nfunc HasCloseElements(numbers []float64, threshold float64) bool {\n',
 'import': 'import (\n    "math"\n)\n',
 'docstring': '// Check if in given list of numbers, are any two numbers closer to each other than given threshold.\n// >>> HasCloseElements([]float64{1.0, 2.0, 3.0}, 0.5)\n// false\n// >>> HasCloseElements([]float64{1.0, 2.8, 3.0, 4.0, 5.0, 2.0}, 0.3)\n// true\n',
 'declaration': '\nfunc HasCloseElements(numbers []float64, threshold float64) bool {\n',
 'canonical_solution': '    for i := 0; i < len(numbers); i++ {\n        for j := i + 1; j < len(numbers); j++ {\n            var distance float64 = math.Abs(numbers[i] - numbers[j])\n            if distance < threshold {\n   

In [493]:
# for snippet in go_content:
#     import_string = snippet['import']
#     prompt = snippet['prompt']
#     prompt = prompt.replace(import_string, '')

#     prompt = ''.join(prompt.split('func ')[:-1])

#     print(prompt)
#     print('--x---x---x--\n')

#### Java

In [494]:
def remove_unindented(lines, full_func):
    while (not lines[0]) or (lines[0] == lines[0].lstrip()):
        full_func = True
    return lines, full_func

def remove_signature_java(lines, full_func):
    '''
    deals with three cases
    a) first lines are comments, before method header
    b) first line is method-header
    c) b), but with incorrect indentation? so header had accidentally been removed?
    '''
    if not (full_func or lines[0].lstrip()[:6] == 'public'):
        return lines
    
    removed = False
    og_lines = copy.deepcopy(lines)
    
    while not removed:
        if lines == []:
            return og_lines
        
        line = lines.pop(0)
        if line.lstrip()[:6] == 'public':
            removed = True
        
    return lines

def balance_brackets_java(processed):
    difference = processed.count('{') - processed.count('}') + 2
    assert difference in [0, 1, 2], 'brackets ain\'t balancing'
    return processed + ('}' * difference)

def find_code_java(completion: str, lang) -> str:
    code = identify_codeblock(completion)
    lines = code.splitlines()
    lines, full_func = remove_unindented(lines, False)
    lines = remove_signature_java(lines, full_func)

    processed = '\n'.join(lines)
    processed = balance_brackets_java(processed)
    
    return processed

In [495]:
def get_final_java(state, completion):
    final_code = state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
    return final_code

#### Java [archived]

In [496]:
# def remove_method_header(lines,):
#     '''
#     deals with three cases
#     a) first lines are comments, before method header
#     b) first line is method-header
#     c) b), but with incorrect indentation? so header had accidentally been removed?
#     '''
#     removed = False
#     og_lines = copy.deepcopy(lines)
    
#     while not removed:
#         if lines == []:
#             return og_lines
        
#         line = lines.pop(0)
#         if line.lstrip()[:6] == 'public':
#             removed = True
        
#     return lines

# def balance_brackets(lines):
#     final_code = '\n'.join(lines)
#     difference = final_code.count('{') - final_code.count('}') + 2
#     assert difference in [0, 1, 2], 'brackets ain\'t balancing'
#     return final_code + ('}' * difference)

# def remove_unindented(lines, full_func):
#     import_statements = []
#     while (not lines[0]) or (lines[0] == lines[0].lstrip()):
#         line = lines.pop(0)
#         full_func = True
#         if 'import' in line: import_statements.append(line)
    
#     return lines, import_statements, full_func

# def find_code_java(completion: str) -> str:
#     code = identify_codeblock(completion)
#     lines = code.splitlines()

#     lines, import_statements, full_func = remove_unindented(lines, False)

#     if full_func or lines[0].lstrip()[:6] == 'public':
#         lines = remove_method_header(lines)

#     processed_completion = balance_brackets(lines)
#     import_statements = '\n'.join(import_statements)
    
#     return processed_completion, import_statements

In [497]:
# def get_final_java(imports, state, completion):
#     final_code = imports + '\n' + state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
#     return final_code

#### JavaScript

In [498]:
def remove_signature_js(code: str) -> str:
    lines = code.splitlines()
    found = False
    for line in lines:
        if line.lstrip().startswith('const'):
            lines.remove(line)
            found = True
            break  

    if not found: 
        print('error extracting function body')
        return 'errormsg'
    return "\n".join(lines)

def balance_brackets_js(completion):
    difference = completion.count('{') - completion.count('}') 
    if difference == 0:
        return completion + '};'
    elif difference == -1:
        return completion
    else:
        print('weird formatting')
        return completion

async def find_code_js(completion: str, lang) -> str:
    processed = await remove_signature(completion, lang)
    processed = identify_codeblock(processed)
    # processed = balance_brackets_js(processed)
    # processed = remove_signature_js(processed)
    return processed

In [499]:
# def get_final_js(state, completion):
#     final_code = state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
#     return final_code

def get_final_js(state, completion):
    prompt = state.metadata['prompt']
    prompt = ''.join(prompt.split('const ')[:-1])
    final_code = prompt + completion + "\n\n" + state.metadata['test'] + "\n"
    return final_code

#### Python

In [500]:
def remove_signature_python(code: str) -> str:
    try:
        tree = ast.parse(code)
        for node in tree.body:
            if not isinstance(node, ast.FunctionDef):
                continue
            code_lines = code.splitlines()
            start = node.body[0].lineno - 1
            end = node.body[-1].end_lineno
            body_lines = code_lines[start:end]
            return "\n".join(body_lines)

    except Exception as e:
        print(f"Error extracting function body: {e}")
        return "errormsg"
    
async def find_code_python(completion, lang):
    processed = await remove_signature(completion, lang)
    processed = identify_codeblock(processed)

    # processed = remove_signature_python(processed)
    return processed

In [501]:
# def get_final_python(state, completion):
#     imports = "\n".join(IMPORT_HELPER["python"]) + "\n"
#     final_code = imports + state.metadata['prompt'] + completion + "\n" + state.metadata['test'] + "\n"
#     return final_code

def get_final_python(state, completion):
    imports = "\n".join(IMPORT_HELPER["python"]) + "\n"
    prompt = state.metadata['prompt']
    prompt = ''.join(prompt.split('def ')[:-1])
    final_code = imports + prompt + completion + "\n" + state.metadata['test'] + "\n"
    return final_code

### Routing

In [478]:
async def get_final(state, lang, task_id):
    model_completion = state.output.completion

    process_code = globals()[f'find_code_{lang}']
    processed_completion = await process_code(model_completion, lang)

    final = globals()[f'get_final_{lang}']
    final_code = final(state, processed_completion)

    if 'errormsg' in final_code:
        print(f'error in sample: {task_id}')
    
    return model_completion, processed_completion, final_code

### Scorers

In [479]:
async def python_scorer(final_code, *args):
    try:
        result = await sandbox().exec(
            cmd=["python", "-c", final_code],
            timeout=30,
        )
    except TimeoutError:
        result = ExecResult(False, 1, "", "Verification timed out.")
    
    return result

In [480]:
async def js_scorer(final_code, *args):
    try:
        result = await sandbox().exec(
            cmd=["node", "-e", final_code],
            timeout=30,
        )
    except TimeoutError:
        result = ExecResult(False, 1, "", "Verification timed out.")

    return result

In [481]:
async def go_scorer(final_code, idx, tmp_dir):
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir, exist_ok=True)
    file = os.path.join(tmp_dir, 'main_test.go')

    with contextlib.chdir(tmp_dir):
        open(file, 'w').write(final_code)
        if not os.path.exists('go.mod'):
            try:
                subprocess.run(
                    ['/usr/local/go/bin/go', 'mod', 'init', f'example.com/tmpmod_{idx}'],
                    check=True, 
                    capture_output=True,
                )
            except subprocess.CalledProcessError as e:
                print("Error running go mod init:")
                print(e.stderr)
        subprocess.run(
            ['/usr/local/go/bin/go', 'mod', 'tidy'], 
            check=True, 
            capture_output=True,
        )

    try:
        result = await sandbox().exec(
            cmd=["/usr/local/go/bin/go", "test", file],
            timeout=30,
            cwd=tmp_dir
        )
    except TimeoutError:
        result = ExecResult(False, 1, "", "Verification timed out.")

    shutil.rmtree(tmp_dir)

    return result

In [482]:
async def java_scorer(final_code, idx, tmp_dir):
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir, exist_ok=True)
    file = os.path.join(tmp_dir, 'Main.java')

    with contextlib.chdir(tmp_dir):
        open(file, 'w').write(final_code)
    
    try:
        compile_proc = subprocess.run(
            ["javac", "Main.java"],
            cwd=tmp_dir,
            capture_output=True,
            text=True,
            timeout=30  
        )
        if compile_proc.returncode != 0:
            print(f"Compilation failed! Idx: {idx}")
            print("stderr:", compile_proc.stderr)
    except subprocess.TimeoutExpired:
        print("Compilation timed out!")
    except Exception as e:
        print("Compilation error:", e)

    try:
        result = await sandbox().exec(
            cmd=["java", "-cp", tmp_dir, "Main"],
            timeout=30,
        )
    except TimeoutError:
        result = ExecResult(False, 1, "", "Verification timed out.")

    shutil.rmtree(tmp_dir)

    return result

In [483]:
async def cpp_scorer(final_code, idx, tmp_dir):
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir, exist_ok=True)
    file = os.path.join(tmp_dir, 'test.cpp')
    executable = os.path.join(tmp_dir, 'test.out')

    open(file, 'w').write(final_code)
    
    try:
        compile_proc = subprocess.run(
            ["g++", "-std=c++17", file, "-o", executable, '-lssl', '-lcrypto'],
            capture_output=True,
            text=True,
            timeout=30  # seconds, adjust as needed
        )
        if compile_proc.returncode != 0:
            print(f"Compilation failed! task number: {idx}")
            # print("stderr:", compile_proc.stderr)
    except subprocess.TimeoutExpired:
        print("Compilation timed out!")

    if os.path.exists(executable):
        try:
            result = await sandbox().exec(
                cmd=[executable],
                timeout=30
            )
        except TimeoutError:
            result = ExecResult(False, 1, "", "Verification timed out.")
        except Exception as e:
            print(f'execution failed cuz of: {e}')
    else:
        result = ExecResult(False, 1, "", "Compiler Error")

    shutil.rmtree(tmp_dir)

    return result

### Inspect Pipeline

In [484]:
def get_lang_idx(task_id: str):
    lang, idx = task_id.split('/')
    lang = lang.lower()
    if lang == 'javascript':
        lang = 'js'
    idx = int(idx)

    return lang, idx

@scorer(metrics=[accuracy(), stderr()])
def main_scorer() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        task_id = state.sample_id
        lang, idx = get_lang_idx(task_id)

        model_completion, processed_completion, final_code = await get_final(state, lang, task_id)

        tmp_dir = f'/root/srf-project/test_humaneval-x/tmp/test_{idx}/'
        my_scorer = globals()[f'{lang}_scorer']
        result = await my_scorer(final_code, idx, tmp_dir)

        success = result.success and (result.stderr == '')
        
        return Score(
            value=CORRECT if success else INCORRECT,
            explanation="".join(
                ["The following verification code was executed:\n\n"]
                + [final_code]
                + [f"\nThe submission was incorrect\n\n{result.stderr}"]
                if not result.success
                else [""]
            ),
            metadata={
                'completion': model_completion,
                'processed': processed_completion,
                'final_code': final_code,
                'idx': idx,
                'task_id': task_id,
            },
        )

    return score

In [485]:
language = 'go'

def humaneval_record_to_sample(record):
    model_input = HUMANEVAL_INSTRUCTION + LANG_PREFIX[language] + '\n' + record['prompt'] 

    idx = int(record['task_id'].split('/')[-1])

    metadata = {
        "prompt": record["prompt"],
        "test": record["test"],
        "declaration": record["declaration"]
    }
    if language == 'go':
        metadata['import'] = go_content[idx]['import']
        metadata['test_setup'] = go_content[idx]['test_setup']
    
    return Sample(
        id=record["task_id"],
        input=model_input,
        target=record["canonical_solution"],
        metadata=metadata,
    )

humaneval_dataset = hf_dataset(
    path = 'THUDM/humaneval-x',
    name = language,
    split = 'test',
    sample_fields = humaneval_record_to_sample,
    trust = True,
)

In [486]:
samples = 30

@task
def humaneval():
    return Task(
        dataset = humaneval_dataset[-samples:],
        solver = generate(),
        scorer = main_scorer(),
        sandbox = 'local',
    )

## Experimentation

In [502]:
epochs = 1
model = llama_model

inspect_ai.eval(
    humaneval(), 
    model = model, 
    epochs = epochs,
    log_dir = '/root/srf-project/test_humaneval-x/pipeline_check/go/'
)

Output()

In [ ]:
path = '/root/srf-project/test_humaneval-x/pipeline_check/python/llama_8b.eval'
log = read_eval_log(path)

def eval_duration(log):
    start_time = log.stats.started_at
    start_time = start_time.split('T')[-1].split('+')[0]

    end_time = log.stats.completed_at
    end_time = end_time.split('T')[-1].split('+')[0]
    
    fmt = "%H:%M:%S"
    t1 = datetime.strptime(start_time, fmt)
    t2 = datetime.strptime(end_time, fmt)

    duration = (t2-t1).seconds

    return f'{duration} seconds'

def eval_score(log):
    score = log.results.scores[0].metrics['accuracy'].value
    return f'{score * 100:.2f}%'
    
print(eval_duration(log))
print(eval_score(log))